## Excersize 3

In [2]:
using Roots,Distributions,Parameters,BasisMatrices
@with_kw mutable struct TaxParams
    σ::Float64 = 2. #Standard
    γ::Float64 = 2. #Targets Frisch elasticity of 0.5
    σ_α::Float64 = sqrt(0.147) #Taken from HSV
    N::Int64 = 1000
    alphaDist::Distribution = Normal(-σ_α^2/2,σ_α)
    αvec::Vector{Float64} = rand(alphaDist,N)
end
params = TaxParams()

TaxParams
  σ: Float64 2.0
  γ: Float64 2.0
  σ_α: Float64 0.38340579025361626
  N: Int64 1000
  alphaDist: Normal{Float64}
  αvec: Array{Float64}((1000,)) [-0.07494931334069896, -0.7951433894129252, -0.2963813570687809, -0.5203570149299562, -0.708993048089964, 0.22761345497944485, 0.16190034743208634, 0.11443901857031341, 0.19487791252305486, -0.7741346441784632  …  -0.32275155893724, 0.4997499959471932, -0.5111566459216346, 0.38478439229799927, 0.38995043513130423, -0.23923020645477752, 0.29620766099549306, 0.35220045053415727, -0.2753249510472501, -0.28623280956510916]


## Interpolating the HH policy functions
* Try using Chebyshev polynomials

In [ ]:
"""
    approximate_household_labor(params,NlŴ,NT)

Approximates HH policies as a function of log after tax wage and transfers.
"""
function approximate_household_labor(params,NlŴ,NT)
    @unpack σ_α,σ,γ = params
    lŴbasis = ChebParams(NlŴ,-5*σ_α+log(1-.8),5*σ_α)
    Tbasis = ChebParams(NT,0.,2.) #we know optimal tax will always be positive
    basis = Basis(lŴbasis,Tbasis)
    X = nodes(basis)[1]
    N = size(X,1) #How many nodes are there?
    c,h = zeros(N),zeros(N)
    for i in 1:N 
        Ŵ,T = exp(X[i,1]),X[i,2]
        res(h) = (Ŵ*h+T)^(-σ)*Ŵ-h^γ
        min_h = max(0,(.0000001-T)/Ŵ) #ensures c>.0001
        h[i] = fzero(res,min_h,20000.) #find hours that solve HH problem
        c[i] = Ŵ*h[i]+T
    end
    U = @. c^(1-σ)/(1-σ)-h^(1+γ)/(1+γ)
    return (cf=Interpoland(basis,c),hf=Interpoland(basis,h),Uf=Interpoland(basis,U))
end;

## Government Welfare and Budget Constraint
* Adjust Government budget constraint to us pre-stored functions

In [ ]:
"""
    budget_residual(params,policies,τ,T)

Computes the residual of the HH budget constraint given policy (τ,T)
"""
function budget_residual(params,policies,τ,T)
    @unpack αvec = params
    @unpack hf = policies
    N = length(αvec)
    X = [αvec .+ log(1-τ)  T*ones(N)]
    tax_income = sum(hf(X).*exp.(αvec).*τ)/N
    return tax_income - T
end;

* Same with objective function

In [ ]:
"""
    government_welfare(params,policies,τ,T)

Solves for government welfare given tax rate τ
"""
function government_welfare(params,policies,τ,T)
    @unpack αvec = params
    @unpack Uf = policies
    N = length(αvec)
    X = [αvec .+ log(1-τ)  T*ones(N)]
    return sum(Uf(X))/N
end;

## Solving For Optimal Policy
* Wrap NLopt in a function for ease of use

In [ ]:
using NLopt
""" 
    find_optimal_policy(αvec,Uf,hf)

Computes the optimal policy given policy fuctions hf and indirect utility Uf
"""
function find_optimal_policy(params,policies)
    @unpack αvec = params
    @unpack Uf,hf = policies
    opt = Opt(:LN_COBYLA, 2)
    lower_bounds!(opt, [0., 0.])
    upper_bounds!(opt, [0.5,Inf])
    ftol_rel!(opt,1e-8)

    min_objective!(opt, (x,g)->-government_welfare(params,policies,x[1],x[2]))
    equality_constraint!(opt, (x,g) -> -budget_residual(params,policies,x[1],x[2]))

    minf,minx,ret = NLopt.optimize(opt, [0.3, 0.3])
    if ret == :FTOL_REACHED
        return minx
    end
end;

## Let's try this out

In [ ]:
policies = approximate_household_labor(params,10,10)
find_optimal_policy(params,policies)
@time find_optimal_policy(params,policies)
#Remember the old code took 2 seconds

  0.012600 seconds (16.66 k allocations: 48.834 MiB, 21.09% gc time)


2-element Vector{Float64}:
 0.3612118543260959
 0.3177638391495579

## How Does the Policy Depend On the Degree of Approximation?

In [ ]:
policies = approximate_household_labor(params,5,5)
find_optimal_policy(params,policies)

2-element Vector{Float64}:
 0.41625303281897674
 0.35949657855401695

In [ ]:
policies = approximate_household_labor(params,10,10)
find_optimal_policy(params,policies)

2-element Vector{Float64}:
 0.3612118543260959
 0.3177638391495579

In [ ]:
policies = approximate_household_labor(params,20,20)
find_optimal_policy(params,policies)

2-element Vector{Float64}:
 0.36142838596726296
 0.317927168481032

* Big difference going from 5 degree to 10 degree
    * But 10 to 20 degree only effects the 4th decimal place